In [1]:
# Cell 1: Imports
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', 200)


In [2]:
# Cell 2: Load main table
APP = pd.read_csv('../data/raw/application_train.csv')
print("Application:", APP.shape)
APP.head()


Application: (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [3]:
# Cell 3: Basic cleaning - drop anomalies
APP = APP[APP['CODE_GENDER'] != 'XNA']
APP['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)


C:\Users\99sma\AppData\Local\Temp\ipykernel_5236\3330532525.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  APP['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)


In [4]:
# Cell 5: Missing values overview
miss = APP.isnull().mean().sort_values(ascending=False)
miss[miss > 0].head(10)


COMMONAREA_AVG              0.698722
COMMONAREA_MODE             0.698722
COMMONAREA_MEDI             0.698722
NONLIVINGAPARTMENTS_MEDI    0.694332
NONLIVINGAPARTMENTS_MODE    0.694332
NONLIVINGAPARTMENTS_AVG     0.694332
FONDKAPREMONT_MODE          0.683864
LIVINGAPARTMENTS_AVG        0.683552
LIVINGAPARTMENTS_MEDI       0.683552
LIVINGAPARTMENTS_MODE       0.683552
dtype: float64

In [5]:
# Cell 6: Feature engineering (Kaggle best practices)
APP['AGE_YEARS'] = -APP['DAYS_BIRTH'] / 365
APP['EMPLOYMENT_YEARS'] = -APP['DAYS_EMPLOYED'] / 365
APP['CREDIT_INCOME_RATIO'] = APP['AMT_CREDIT'] / APP['AMT_INCOME_TOTAL']
APP['ANNUITY_INCOME_RATIO'] = APP['AMT_ANNUITY'] / APP['AMT_INCOME_TOTAL']
APP['GOODS_CREDIT_RATIO'] = APP['AMT_GOODS_PRICE'] / APP['AMT_CREDIT']
APP['PAYMENT_LENGTH'] = APP['AMT_CREDIT'] / APP['AMT_ANNUITY']


In [6]:
# Cell 7: Handle EXT_SOURCE imputation + derived features
for col in ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']:
    APP[col + '_imp'] = APP[col].fillna(APP[col].median())
APP['EXT_SOURCES_MEAN'] = APP[['EXT_SOURCE_1_imp','EXT_SOURCE_2_imp','EXT_SOURCE_3_imp']].mean(axis=1)
APP['EXT_SOURCES_STD'] = APP[['EXT_SOURCE_1_imp','EXT_SOURCE_2_imp','EXT_SOURCE_3_imp']].std(axis=1)
APP['EXT_SOURCES_PROD'] = APP[['EXT_SOURCE_1_imp','EXT_SOURCE_2_imp','EXT_SOURCE_3_imp']].prod(axis=1)


In [7]:
# Cell 8: Encode categoricals
le = LabelEncoder()
for c in APP.select_dtypes('object'):
    if APP[c].nunique() == 2:
        APP[c] = le.fit_transform(APP[c])
APP = pd.get_dummies(APP, drop_first=True)


In [9]:
# Cell 9: Load bureau and aggregate
bureau = pd.read_csv('../data/raw/bureau.csv')
bb = pd.read_csv('../data/raw/bureau_balance.csv')
# Map status values to numerical
status_map = {
    '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5,
    'C': 0,  # Closed treated as no delay
    'X': 0   # No DPD (days past due)
}
bb['STATUS'] = bb['STATUS'].map(status_map)
bb_agg = bb.groupby('SK_ID_BUREAU')['STATUS'].agg(['mean','size']).rename(columns={'mean':'BB_STATUS_MEAN','size':'BB_COUNT'})
bureau = bureau.join(bb_agg, on='SK_ID_BUREAU')
bb_agg = bureau.groupby('SK_ID_CURR').agg({
    'DAYS_CREDIT':'min','DAYS_CREDIT_ENDDATE':'max',
    'CREDIT_DAY_OVERDUE':'mean','AMT_CREDIT_SUM':'sum',
    'BB_STATUS_MEAN':'mean','BB_COUNT':'sum'})
APP = APP.join(bb_agg, on='SK_ID_CURR')
del bureau, bb, bb_agg; gc.collect()


3097

In [10]:
# Cell 10: Previous applications
prev = pd.read_csv('../data/raw/previous_application.csv')
prev_agg = prev.groupby('SK_ID_CURR').agg({
    'AMT_APPLICATION':'mean','AMT_CREDIT':'mean',
    'AMT_DOWN_PAYMENT':'mean','DAYS_DECISION':'min',
    'CNT_PAYMENT':'sum','NAME_CONTRACT_TYPE':'nunique'})
prev_agg.columns = [f'PREV_{c}' for c in prev_agg.columns]
APP = APP.join(prev_agg, on='SK_ID_CURR')
del prev, prev_agg; gc.collect()


0

In [12]:
# Cell 11: Final row-wise cleaning before imputation

# Drop rows with more than 100 missing values
APP = APP.dropna(thresh=APP.shape[1] - 100)

# Then perform median imputation for remaining missing values
APP.fillna(APP.median(), inplace=True)

# Shape after cleaning
APP.shape


(307507, 253)

In [15]:
# Cell 12: Split train/test, save processed
y = APP['TARGET']
X = APP.drop(['TARGET','SK_ID_CURR'], axis=1)
X.to_csv('../data/processed/X_all.csv', index=False)
y.to_csv('../data/processed/y_all.csv', index=False)
